# RNA-Seq Analysis using Snakemake and Google Cloud Life Sciences API

## Overview

This short tutorial demonstrates how to run an RNA-Seq workflow using a prokaryotic data set. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to quantitate gene expression. This tutorial uses a popular workflow manager called [Snakemake](https://snakemake.readthedocs.io/en/stable/) run via the [Google Cloud Life Sciences API](https://cloud.google.com/life-sciences/docs/reference/rest).

### Step 1: Create a new GS Bucket to store input and output files
Note that your bucket has to be globally unique, so make sure you don't just copy the example here or it won't work

In [ ]:
#change this bucket name
%env BUCKET=<YourBucketName>

In [ ]:
#will only create the bucket if it doesn't yet exist
!gsutil ls gs://$BUCKET >& /dev/null || gsutil mb gs://$BUCKET

In [ ]:
#set versioning on the bucket so it can overwrite old files
!gsutil versioning set on gs://$BUCKET

### STEP 2: Install mambaforge and snakemake
First install mambaforge, then use mamba to install snakemake.

In [ ]:
! curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
! bash Mambaforge-$(uname)-$(uname -m).sh -b -p $HOME/mambaforge

In [ ]:
#add to your path
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/Mambaforge/bin"

In [ ]:
#install snakemake
! mamba install -y -c conda-forge -c bioconda snakemake

### STEP 3: Copy FASTQ Files
In order for this tutorial to run quickly, we will only analyze 50,000 reads from a sample from both sample groups instead of analyzing all the reads from all six samples. These files have been posted on a Google Storage Bucket that we made publicly accessible.

In [ ]:
! gsutil -m cp -r gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/ gs://$BUCKET/data/raw_fastq

Create a fake path to data/fastqc so that snakemake can write files to that bucket path, otherwise the pipeline crashes.

In [ ]:
! touch blank.txt
! gsutil cp blank.txt gs://$BUCKET/data/fastqc/

### STEP 4: Copy reference files that will be used by Salmon
Salmon is a tool that aligns RNA-Seq reads to a set of transcripts rather than the entire genome.

In [ ]:
!gsutil -m cp -r gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_transcripts.fasta gs://$BUCKET/data/reference/M_chelonae_transcripts.fasta
!gsutil -m cp -r gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/data/reference/decoys.txt gs://$BUCKET/data/reference/decoys.txt


### STEP 5: Copy data file for Trimmomatic

In [ ]:
!gsutil -m cp gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa gs://$BUCKET/TruSeq3-PE.fa

### STEP 6: Copy data and config files that will be used in our snakemake environment

Next download config files for our snakemake environment.

In [ ]:
!gsutil -m cp -r gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/envs/ .
!gsutil -m cp gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/config.yaml .
!gsutil -m cp gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/snakefile_ls_api .

Add the bucket path to the end of your config file. Since this file was written for running snakemake locally we have to make a few edits to run on the LS API.

In [ ]:
!echo 'bucket:' >> config.yaml

In [ ]:
!echo '   '$BUCKET >>config.yaml

Add bucket path to the snakefile

In [ ]:
!sed -i 's/print(SAMPLES)/BUCKET=config["bucket"]/' snakefile_ls_api

### Step 7: Set up your local environment
You need to generate a [service account key](https://cloud.google.com/iam/docs/creating-managing-service-account-keys) for the compute engine default service account to interact with the Life Sciences API using Snakemake. Download the key and copy it to this VM. Then assign the path of the json file to an environment variable.

In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS=cloud_creds.json

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

Set your project (make sure to replace $Project with your project name)

In [ ]:
!gcloud config set project $PROJECT

Initialize a local git repo

In [ ]:
!git init

Configure conda

In [ ]:
conda config --set channel_priority strict

### STEP 8: Run snakemake using the Life Sciences API

Aside from the .yaml config files which information about software, dependencies, and versions -- snakemake uses a snakefile which contains information about a workflow.

This can be a powerful tool as it allows one to operate and think in terms of workflows instead of individual steps. You should open the snakefile to look at it further. It is composed of 'rules' we have created. Snakefiles work largely based on inputs. For a given input/output, there is an associated 'rule' that runs. Snakefiles may take a while to get the idea of what's going on, but in simplest terms here we take an input of .fastq files, and based on the snakefile rules we created, those fastq files are run through the entire workflow. The rule_all at the top determines which rules are run based on the input files for rule_all (which are outputs from the target rules. Comment out rules you don't want to run. 

Snakemake requires that you have a service account key to authenticate with the Life Sciences API. This actually is not necessary to use the API from within a notebook, but Snakemake does require it since Snakemake is expecting you to run the command from your own terminal using the SDK. To see all the commands you can run with Snakemake via the Life Sciences API, check out the [docs](https://snakemake.readthedocs.io/en/stable/executor_tutorial/google_lifesciences.html).

Now we can run the Life Sciences APi. You will see that each rule is submitted as a separate job. If the pipeline crashes, the way to troubleshoot is by reading the API logs, or the snakemake rule logs (same info). You can find the Life Sciences API logs by pasting in the gcloud command given in yellow.

For example: 
```
gcloud beta lifesciences operations describe <JOB_ID>
```
Or you can view the logs by finding the path given for logs, and then use gsutil to copy that file locally, or go to the bucket and double click the file. You can get the job ID for the output file in the green section of the rule print out.

In [ ]:
%%time
! snakemake --forceall --snakefile snakefile_ls_api --google-lifesciences --default-remote-prefix $BUCKET --use-conda --google-lifesciences-region us-central1 -j 24 --rerun-incomplete --default-resources "machine_type=n2-standard"

### STEP 9: Report the top 10 most highly expressed genes in the samples.

Top 10 most highly expressed genes in the wild-type sample. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`


In [ ]:
!gsutil rm gs://$BUCKET/data/quants/SRR13349122_quant
!gsutil rm gs://$BUCKET/data/quants/SRR13349128_quant

In [ ]:
!gsutil ls gs://$BUCKET/data/quants/*

In [ ]:
!gsutil cp -r gs://$BUCKET/data/quants/SRR13349122_quant/ .
!gsutil cp -r gs://$BUCKET/data/quants/SRR13349128_quant/ .

In [ ]:
!sort -nrk 5,5 SRR13349122_quant/quant.sf | head -10

Top 10 most highly expressed genes in the double lysogen sample.


In [ ]:
!sort -nrk 5,5 SRR13349128_quant/quant.sf | head -10

### STEP 10: Report the expression of a putative acyl-ACP desaturase (BB28_RS16545) that was downregulated in the double lysogen relative to wild-type
A acyl-transferase was reported to be downregulated in the double lysogen as shown in the table of the top 20 upregulated and downregulated genes from the paper describing the study.
![RNA-Seq workflow](images/table-cushman.png)

Use `grep` to report the expression in the wild-type sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [ ]:
!grep 'BB28_RS16545' SRR13349122_quant/quant.sf

Use `grep` to report the expression in the double lysogen sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [ ]:
!grep 'BB28_RS16545' SRR13349128_quant/quant.sf